# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# !pip install snorkel

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('../all_sqlbelle_youtube_comments_4200rows.csv')

In [ ]:
df_train.textDisplay[:-3:-1]

4201                              Thank you George :)
4200    Awesome! Looking forward to many more videos.
Name: textDisplay, dtype: object

In [ ]:
df_train.tail(2)

,response_kind,vid_channel_id,vid_id,vid_published_at,vid_title,vid_desc,vid_chan_title,is_live_brodcast,vid_publish_time,view_count,...,canRate,viewerRating,likeCount,publishedAt,updatedAt,tcomment_id,reply_id,canReply,totalReplyCount,isPublic
4200,youtube#commentThread,NaN,ZxcWBnXY8ps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,none,1.0,2020-09-02T06:16:13Z,2020-09-02T06:16:13Z,UgzO3LCmld1HWW3XpNt4AaABAg,NaN,True,1.0,True
4201,youtube#comment,NaN,ZxcWBnXY8ps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,none,0.0,2020-09-03T05:16:12Z,2020-09-03T05:16:12Z,UgzO3LCmld1HWW3XpNt4AaABAg,UgzO3LCmld1HWW3XpNt4AaABAg.9D5QmTk8TpI9D7thkihuok,NaN,NaN,NaN


#### Create simple labels to start

In [ ]:
abstain = -1
not_spam = 0
spam = 1

#### Key Word:  simple function to identify comments using "my"

In [ ]:
from snorkel.labeling import labeling_function


@labeling_function()
def lf_keyword_my(x):
    """Many spam comments talk about 'my channel', 'my video', etc."""
    return spam if "my" in x.text.lower() else abstain

#### Regular Expression: simple fuction

In [ ]:
import re


@labeling_function()
def lf_regex_check_out(x):
    """Spam comments say 'check out my video', 'check it out', etc."""
    return spam if re.search(r"check.*out", x.text, flags=re.I) else abstain

#### Arbitrary Heuristics

In [ ]:
@labeling_function()
def lf_short_comment(x):
    """Non-spam comments are often short, such as 'cool video!'."""
    return not_spam if len(x.text.split()) < 5 else abstain

#### Third Party Models:
>
    -We use a third-party sentiment classification model, TextBlob.

    -We combine this with the heuristic that non-spam comments are often positive.

In [ ]:
# !pip install textblob

In [ ]:
from textblob import TextBlob


@labeling_function()
def lf_textblob_polarity(x):
    
    return not_spam if TextBlob(x.text).sentiment.polarity > 0.3 else abstain

# Combining & Cleaning the Labels

In [ ]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

In [ ]:
df = df
#create toy dataset
df =df[['textDisplay']][:100]
#drop na's
df = df.loc[~df.textDisplay.isna()]
# create string and 
df = df.textDisplay.astype(str)

df = df.reset_index()
#rename columns
df.rename(columns = {'index': 'num','textDisplay':'text'},inplace=True)
df.tail(2)

df = df_train
df_train.head(3)

NameError: name 'df' is not defined

In [ ]:
# Define the set of labeling functions (LFs)
lfs = [lf_keyword_my, lf_regex_check_out, lf_short_comment, lf_textblob_polarity]

# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()